# Agent Tools and Configurations

Tools give agents ability to take actions: sequential/parallel calls, error handling, and state persistence.

**What you'll learn:**
- Tools extend agent capabilities with actions
- @wrap_tool_call handles errors gracefully
- Agent selects tools based on docstrings
- Sequential vs parallel execution determined by model
- Clear tool descriptions guide selection

In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from scripts import base_tools

In [3]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash', include_thoughts=False)

In [4]:
# Custom Tool Name and Description
base_tools.web_search.name

base_tools.web_search.description

'Perform a live web search using Ollama Cloud Web Search API for real-time information and news.\n\nInput:\n    query: search query string\n\nOutput:\n    JSON string of top results (max_results=2).'

## Basic Agent with Tools

In [5]:
agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather]
)

response = agent.invoke({
    'messages': [HumanMessage("What's the weather in Tokyo?")]
})

response['messages'][-1].text

'The weather in Tokyo is Sunny with a temperature of 15°C. The wind is from the SW at 39.6 kph, and the humidity is 36%.'

In [6]:
# Agent without tools - single LLM node
agent = create_agent(model=model, tools=[])

response = agent.invoke({
    'messages': [HumanMessage("Hello")]
})

response['messages'][-1].text

'Hello! How can I help you today?'

## Sequential Tool Calls

In [7]:
# Agent calls tools in sequence
response = agent.invoke({
    'messages': [HumanMessage(
        "Search for Apple news, then tell me weather in Cupertino"
    )]
})

response['messages'][-1].text

"Here's a summary of recent Apple news:\n\nApple recently held its **Worldwide Developers Conference (WWDC 2024)**, where it unveiled significant updates across its platforms:\n\n*   **Apple Intelligence:** A new personal AI system deeply integrated into iOS 18, iPadOS 18, and macOS Sequoia, focusing on privacy, personalization, and productivity features like enhanced writing tools, image generation, and a more powerful Siri.\n*   **iOS 18:** Major updates including home screen customization, a redesigned Control Center, a new Photos app experience, and enhancements to Messages.\n*   **macOS Sequoia:** Introduces iPhone Mirroring, new Safari features, and gaming improvements.\n*   **visionOS 2:** Updates for Apple Vision Pro, including new spatial photos capabilities and gestures.\n*   **watchOS 11:** Features a new Vitals app, training load measurement, and customizable Smart Stack.\n*   **Vision Pro International Expansion:** Apple Vision Pro is expanding to several new countries, in

## Parallel Tool Calls

In [8]:
# Agent may call tools in parallel (independent requests)
response = agent.invoke({
    'messages': [HumanMessage(
        "What's the weather in Paris and London?"
    )]
})

response['messages'][-1].text

"Here's the current weather for Paris and London:\n\n*   **Paris, France:**\n    It's currently around **15°C (59°F)** with **partly cloudy skies**. There's a light breeze, and humidity is moderate.\n\n*   **London, UK:**\n    It's a bit cooler at around **13°C (55°F)** and currently experiencing **light rain**. Humidity is higher, and there's a moderate breeze."

## Tool Error Handling

In [9]:
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage
from langchain.tools import tool
# Tool WITHOUT internal error handling
@tool
def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b  # This will crash on division by zero

# Middleware catches it and returns graceful error to model
@wrap_tool_call
def handle_tool_errors(request, handler):
    try:
        return handler(request)
    except Exception as e:
        return ToolMessage(
            content=f"Error: {str(e)}. Try different input.",
            tool_call_id=request.tool_call["id"]
        )


agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather, divide],
    middleware=[handle_tool_errors]
)

response = agent.invoke({
    'messages': [HumanMessage("what is 1/0?")]
})

response['messages'][-1].text

'Division by zero is not possible.'

## Accessing Context - State Access

In [ ]:
from langchain.tools import ToolRuntime
from langchain_core.messages import AIMessage
from dataclasses import dataclass

# Tool that retrieves the count of messages in conversation
@tool
def get_message_count(runtime: ToolRuntime) -> str:
    """Get the total number of messages exchanged in the conversation."""
    messages = runtime.state["messages"]

    # Access context as attribute (not dictionary)
    user_id = runtime.context.user_id

    return f"User [{user_id}] history have '{len(messages)}' messages"

@tool
def get_token_count(runtime: ToolRuntime) -> str:
    """Get the actual token usage from AI responses."""
    messages = runtime.state["messages"]
    
    # Access context as attribute (not dictionary)
    user_id = runtime.context.user_id

    # Track token usage from AI messages
    total_input_tokens = 0
    total_output_tokens = 0
    total_tokens = 0
    response_count = 0
    
    for msg in messages:
        if isinstance(msg, AIMessage):
            # Extract usage_metadata if available
            if hasattr(msg, 'usage_metadata') and msg.usage_metadata:
                total_input_tokens += msg.usage_metadata.get('input_tokens', 0)
                total_output_tokens += msg.usage_metadata.get('output_tokens', 0)
                total_tokens += msg.usage_metadata.get('total_tokens', 0)
                response_count += 1
    
    return f"""USER ID: {user_id} 
                AI responses: {response_count}
                Input tokens: {total_input_tokens}
                Output tokens: {total_output_tokens}
                Total tokens: {total_tokens}"""

@dataclass
class UserContext:
    user_id: str

agent = create_agent(
    model=model,
    tools=[get_message_count, get_token_count],
    context_schema=UserContext
)

In [16]:
response = agent.invoke({
    'messages': [
        HumanMessage("how many messages i have?")
    ]
}, context=UserContext(user_id="user_123"))

response

{'messages': [HumanMessage(content='how many messages i have?', additional_kwargs={}, response_metadata={}, id='32c921c2-2a3c-42be-b18c-31320d120dcf'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_message_count', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'5f5b86fa-0e0b-4f0c-995f-969c9eb8e4f4': 'CssBAXLI2nx30fFuy8S3NxLuIIEgsE09JwZAi6IA2ThA8OJ6WT0+oK5uooM76+EPdup57A+nP400r7UstSCXjX23dyct+0V+eEZGVXCaYcXWyDFafYguQ0MMr+razTPCfhcvm4+Z7/uECOcoSDJlEvTE8mrhc3TzYyoPgkiyAzPIGUdm6rjjxxMuPgZURx0BVC4YrVNQrfdcJKKYrGYVaOxOU+zBSLAQScXEVs2fJb20rCx1XP1fG5/oK8rZBYbbenKLoK5jS/+murwQ49k='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bb5fd-f185-79a2-a498-82ffb0054a64-0', tool_calls=[{'name': 'get_message_count', 'args': {}, 'id': '5f5b86fa-0e0b-4f0c-995f-969c9eb8e4f4', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 73

In [12]:
response = agent.invoke({
    'messages': [
        HumanMessage("how many tokens i have?")
    ]
}, context=UserContext(user_id="user_123"))

response

{'messages': [HumanMessage(content='how many tokens i have?', additional_kwargs={}, response_metadata={}, id='b3ad7e73-cba7-4839-84f3-5fe8491bd135'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_token_count', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'5254d736-6d09-4707-9d8d-e3e1fe601340': 'CucBAXLI2nygKX3Ltg95/dQnPv45oYa2TXpynvbNumIMf5uypmg0JjqVYsd772e1wzalNpka1qMhs6gQmSlIprtMzki/sdjFQZKkj8a+9szXmzZuGEjH+ft48pZu8FJSLnDTCvfitWpe/rQqfZN+Us1Vn5TLd9ACjm27vEbMk3eqdbk2yasT8+sG0fb9yRAlhOybquSPFR71BqCzko0Kye7FDdZvV2bRNSpaWtAN6Mlx3EbKB4MwR6epKxLh8c6Q20RhYPglUgy7R34/IrtfYimSLvtq+dmm8YjXE5mdamV+BxbxaXxs9nic'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bb5f1-c026-7812-9522-edf9abda7c75-0', tool_calls=[{'name': 'get_token_count', 'args': {}, 'id': '5254d736-6d09-4707-9d8d-e3e1fe601340', 'type': 'tool_call'}], invalid_tool_calls=[], usag